In [ ]:
!pip install duckduckgo-search
!pip install --upgrade langchain
!pip install langchain_community
import os
import time
from bs4 import BeautifulSoup
import re
import requests
import warnings

import langchain
from langchain.llms import OpenAI
from langchain.agents import load_tools, AgentType, Tool, initialize_agent

warnings.filterwarnings("ignore")
from langchain.llms import Ollama

In [ ]:
llm = Ollama(model="llama3.1:latest", temperature=0)

In [ ]:
# Define custom tools
import yfinance as yf

In [ ]:
def get_stock_price(ticker, history=5):
    if "." in ticker:
        ticker = ticker.split(".")[0]
    ticker = ticker + ".NS"
    stock = yf.Ticker(ticker)
    df = stock.history(period="1y")
    df = df[["Close", "Volume"]]
    df.index = [str(x).split()[0] for x in list(df.index)]
    df.index.rename("Date", inplace=True)
    df = df[-history:]
    return df.to_string()

print(get_stock_price("TITAN"))

In [ ]:
def google_query(search_term):
    if "news" not in search_term:
        search_term = search_term + " stock news"
    url = f"https://www.google.com/search?q={search_term}&cr=countryIN"
    url = re.sub(r"\s", "+", url)
    return url

In [ ]:
def get_recent_stock_news(company_name):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
    g_query = google_query(company_name)
    res = requests.get(g_query, headers=headers).text
    soup = BeautifulSoup(res, "html.parser")
    news = []
    for n in soup.find_all("div", "n0jPhd ynAwRc tNxQIb nDgy9d"):
        news.append(n.text)
    for n in soup.find_all("div", "IJl0Z"):
        news.append(n.text)

    if len(news) > 6:
        news = news[:10]
    news_string = ""
    for i, n in enumerate(news):
        news_string += f"{i}. {n}\n"
    top5_news = "Recent News:\n\n" + news_string

    return top5_news

print(get_recent_stock_news("Asian paints"))

Recent News:

0. Asian Paints share price Today Live Updates : Asian Paints Sees Decline in Stock Performance Today | Stock Market News
1. Asian Paints Hits 52-Week Low at Rs. 2256.9 Amid 33.35% Annual Decline
2. Asian Paints Share Price Highlights: Asian Paints Stock Price History
3. Asian Paints: How Should You Think About the Stock After the Correction? - Profit Hunter by Equitymaster



In [ ]:
def get_financial_statements(ticker):
    if "." in ticker:
        ticker = ticker.split(".")[0]
    ticker = ticker + ".NS"
    company = yf.Ticker(ticker)
    balance_sheet = company.balance_sheet
    if balance_sheet.shape[1] >= 3:
        balance_sheet = balance_sheet.iloc[:, :3]
    balance_sheet = balance_sheet.dropna(how="any")
    balance_sheet = balance_sheet.to_string()
    return balance_sheet

print(get_financial_statements("TATAPOWER.NS"))

In [ ]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

In [ ]:
tools = [
    Tool(
        name="get stock data",
        func=get_stock_price,
        description="Use when you are asked to evaluate or analyze a stock."
                    " This will output historic share price data."
                    " You should input the stock ticker to it."
    ),
    Tool(
        name="DuckDuckGo Search",
        func=search.run,
        description="Use only when you need to get NSE/BSE stock ticker from internet,"
                    " you can also get recent stock related news."
                    " Don't use it for any other analysis or task."
    ),
    Tool(
        name="get recent news",
        func=get_recent_stock_news,
        description="Use this to fetch recent news about stocks."
    ),
    Tool(
        name="get financial statements",
        func=get_financial_statements,
        description="Use this to get financial statement of the company."
                    " With this data company's historic performance can be evaluated."
                    " You should input stock ticker to it."
    )
]

In [ ]:
def Anazlyze_stock(query, company_name, company_ticker):
    Company_name, ticker = company_name, company_ticker
    print({"Query": query, "Company_name": Company_name, "Ticker": ticker})

    stock_data = get_stock_price(ticker, history=10)
    stock_financials = get_financial_statements(ticker)
    stock_news = get_recent_stock_news(Company_name)

    available_information = f"Stock Financials: {stock_financials}\n\nStock News: {stock_news}"

    print("\n\nAnalyzing.....\n")

    analysis = llm(f"Give detailed stock analysis using available data and provide investment recommendation."
                   f" The user is fully aware of investment risk."
                   f" User question: {query}"
                   f" You have the following information available about {Company_name}."
                   f" Write (5-8) pointwise investment analysis to answer user query,"
                   f" At the end conclude with proper explanation."
                   f" Try to give positives and negatives: {available_information}")

    print(analysis)

out=Anazlyze_stock("Is it a good time to invest in Yes Bank?", 'Yes Bank', 'YESBANK')

{'Query': 'Is it a good time to invest in Yes Bank?', 'Company_name': 'Yes Bank', 'Ticker': 'YESBANK'}


Analyzing.....



ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7951db185540>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
Anazlyze_stock("Is it a good time to invest in Vedanta?", 'Vedanta', 'VEDL')

{'Query': 'Is it a good time to invest in Vedanta?', 'Company_name': 'Vedanta', 'Ticker': 'VEDL'}


Analyzing.....



ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7951db3e3820>: Failed to establish a new connection: [Errno 111] Connection refused'))